In [ ]:
def interaction_features(df, feature1, feature2):
    #prefix : 파생변수이름
    df['inter_({})*({})'.format(feature1,
                                feature2)] = df[feature1] * df[feature2]
    df['inter_({})/({})'.format(feature1,
                                feature2)] = df[feature1] / df[feature2]
    return df


def makeInteraction(data):  # interaction of mean and std features
    cnt = 0
    df = data.copy()
    for i in df:
        if ('mean' in i):
            for j in df:
                if ('mean' in j and i < j):
                    interaction_features(df, i, j)
                    cnt += 2
        if ('std' in i):
            for j in df:
                if ('std' in j and i < j):
                    interaction_features(df, i, j)
                    cnt += 2
    print('create {} interaction features\n\n'.format(cnt))
    return df

In [ ]:
df_interaction = makeInteraction(df_4)
df_interaction.dtypes

# df1 - 3

In [ ]:
import datetime
import math

# 하기의 새로운 df를 Part 1.1 내 df로 간주하고 진행할 것.
df = df_test.copy()

#################################################################################
# TODO: Test set 동일한 처리과정 만들기                                         #
#                                                                               
# Part 1.1에서 만든 방법론을 적용 (Null은 없다고 가정 - 이미 처리했음)
# 1. 데이터 타입처리 (계약년월, 건축년도)
# 2. 지역구 단위의 행정구역 새로운 변수 생성 (변수명 '지역구')
# 3. 건물 년식 일 단위로 하여 새로운 변수 생성 (변수명 '건물년식') 
# 4. 평당가격 변수 추가 
# 5. 과제 1에서 만든 새로운 파생 변수 생성  
#################################################################################
import datetime
import math
#################################################################################
# TODO: 데이터 타입 처리                                                       
#################################################################################

#계약년월 to datetime64
def change_type(df_):
#1. 계약 년월, 계약일을 datetime 형식으로 변환 (datetime.datetime 함수 사용)
    contract_yr = []
    for i, j in np.array(df_[['계약년월', '계약일']]):
        contract_yr.append(datetime.datetime(int(str(i)[:4]), int(str(i)[4:6]), j))
    df_['계약년월_adj'] = contract_yr
# 2. 건축년도 nan 값 처리 및 datetime 형식으로 변환 (datetime.datetime 함수사용)
    construct_yr =[]
    for i in np.array(df_['건축년도']):
        if(math.isnan(i)):
            construct_yr.append(i)
        elif(i==0):
            construct_yr.append(np.nan)
        else:
            construct_yr.append(datetime.datetime(int(str(int(i))[:4]),1,1))

    df_['건축년도_adj']=construct_yr

# 3. 거래금액 string -> float 로 형변환 (Dataframe.apply 사용)
    df_['거래금액(만원)_adj']=df_['거래금액(만원)'].str.replace(',', '').astype('float64')
    
    return df_

df = change_type(df)




#################################################################################
# TODO: 합리적인 방법으로 Null 값을 처리                                        #
#################################################################################

def handling_nan(df_):
#1 대지면적 열 삭제
    df_ = df_.drop(['대지면적(㎡)'],axis=1)

#2 '층'과 '도로조건' 비어있는 값들을 최빈값 대체
    df_ = df_.replace({'층': np.nan}, {'층': df_['층'].mode()[0]})
    df_ = df_.replace({'도로조건': np.nan}, {'도로조건': df_['도로조건'].mode()[0]})

#3 건축년도 비어있는 행 제거 : 
    df_ = df_.dropna(subset=['건축년도'])

#4 주요변수 전부 헨들링 되었으므로 그 외 nan이 존재하는 행은 드랍
    df_ = df_.dropna()

    return df_

df_1 = handling_nan(df_1)

#################################################################################
# TODO: 합리적인 방법으로 이상치 값을 처리하고 산점도의 변화 확인               #
#################################################################################
df_2 = df_1.copy()

def remove_outliers(df_):
 #1 %의 이상치를 모두 제거, 층은 큰값 1%제거, 건축년도는 작은값 1% 제거
    lb_1, ub_1 = np.percentile(df_['전용/연면적(㎡)'],[1,99])
    lb_2, ub_2 = np.percentile(df_['층'],[1,99])
    lb_3, ub_3 = np.percentile(df_['건축년도'],[1,99])
    lb_4, ub_4 = np.percentile(df_['거래금액(만원)_adj'],[1,99])

    outliers = np.where((df_['전용/연면적(㎡)']<lb_1)|(df_['전용/연면적(㎡)']>ub_1)
                        |(df_['층']>ub_2)
                        |(df_['건축년도']<lb_3)
                        |(df_['거래금액(만원)_adj']<lb_4)|(df_['거래금액(만원)_adj']>ub_4))
    df_ = df_.drop(df_.index[outliers[0]])

# 2. 도로조건 '-' 제거      
    df_ = df_[df['도로조건'] != '-']

    return df_

df_2 = remove_outliers(df_2)

# 위 df_2를 기반으로 작성할 것
# 층은 큰값 1% 제거, 건축년도는 작은값 1% 

#################################################################################
# TODO: 예측의 유의미한 영향을 미칠 수 있는 파생변수 생성                       #                
#################################################################################
df_3 = df_2.copy()
#Pyeong_1 = 3.305785 #1평 = 약 3.305785제곱미터
Pyeong_1 = 1
def cal_days(start, end): #날짜 차이 일(days) 계산용
    return (start-end).days

import re

def parse_road(string):
    part = string.partition('로')
    road_ =  part[0]+part[1]
    p = re.compile("[^0-9]")
    return ("".join(p.findall(road_)))

def new_variable(df_):
#1 지역구 단위의 행정구역 새로운 변수 생성 (변수명 '지역구') '서울특별시XX구' or '서울특별시 XX구 ', '구로구' 등 예외처리 
    address_ = []
    for i in df_.loc[:, '시군구']:
        index1_ = i.find('서울특별시')
        if i.find('구로구') != -1 :
            index2_ = i.find('구로구')
            address_.append(i[index1_ + 5 : index2_+3].strip())
        else :
            index2_ = i.find('구')
            address_.append(i[index1_ + 5 : index2_+1].strip())
    df_.loc[:, '지역구'] = address_

#2 건물 년식 일 단위로 하여 새로운 변수 생성 (변수명 '건물년식') 
    df_['건물년식'] = df_.apply(lambda x : cal_days(x['계약년월_adj'],x['건축년도_adj']),axis=1)

#3 평당가격 변수 추가 
    df_['평당가격'] = df_['거래금액(만원)_adj']/(df_['전용/연면적(㎡)']/Pyeong_1)

# 4. 그 외의 합리적인 새로운 파생변수 생성    
#도로명의 양식을 통일한 '도로명_adj' 변수 생성
# 도로명의 양식 통일화. 'xxx로xx길'은 'xxx로'로 변경, 'xxx(숫자)길'은 'xxx길'로 변경
#63로와 4.19로는 따로 조건문을 통해 처리
    df_ = df_[df_['도로명'] != '-']
    df_['도로명_adj'] = df_['도로명'].apply(lambda x : parse_road(x))
    road_adj =[]
    for i in np.array(df_['도로명_adj']):
        if(i=='.로'):
            road_adj.append('4.19로')
        elif(i=='로'):
            road_adj.append('63로')  
        else:
            road_adj.append(i)
    df_['도로명_adj'] = road_adj

# 지역구+도로명 주소 파생변수 생성
    df_['지역구_도로명'] = df_['지역구'] + ' ' +df_['도로명_adj']
    

    return df_
                                     
df_3 = new_variable(df_3)
# df_3를 사용하여 작성할 것

# df4 (원핫)

In [25]:



CATEGORICAL = ['용도지역', '건축물주용도', '도로조건','지역구']
df_4 = df_3.loc[:,[ '전용/연면적(㎡)', '층', '건물년식', '평당가격', '거래금액(만원)_adj']]

#################################################################################
# TODO:범주형 변수 처리                                                         #
#################################################################################
df_4 = df_3.loc[:,['전용/연면적(㎡)', '층', '건물년식', '평당가격', '거래금액(만원)_adj',
                   '용도지역','건축물주용도', '도로조건','지역구','지역구_도로명']]


# 범주형 변수처리
# 원핫 인코딩을 하면 dimension이 너무 들어나는 문제, ordinal하게 변수를 1,2,3,4 이런식으로 바꿔주면 
# 모델이 RF로 스케일링에 robust하지만, feature가 정확한 정보를 반영하기 힘들다고 판단
# 각 카테고리 변수의 변수별 평당가격의 평균과 std를 피쳐로써 반영.
# 도로명주소를 ~~~로 까지만 정제한 이후 도로명 주소에 대해서도 각 도로명별 평균과 std반영
# (도로명이 같은 도로가 다른구에도 존재하므로 지역구과 결합하여 반영)
def set_cate(df_):  
    df_road_mean = pd.DataFrame(df_['평당가격'].groupby(df_['도로조건']).mean())
    df_road_std = pd.DataFrame(df_['평당가격'].groupby(df_['도로조건']).std())
    df_usageloc_mean = pd.DataFrame(df_['평당가격'].groupby(df_['용도지역']).mean())
    df_usageloc_std = pd.DataFrame(df_['평당가격'].groupby(df_['용도지역']).std())
    df_usagebuilding_mean = pd.DataFrame(df_['평당가격'].groupby(df_['건축물주용도']).mean())
    df_usagebuilding_std = pd.DataFrame(df_['평당가격'].groupby(df_['건축물주용도']).std())
    df_location_mean = pd.DataFrame(df_['평당가격'].groupby(df_['지역구']).mean())
    df_location_std = pd.DataFrame(df_['평당가격'].groupby(df_['지역구']).std())
    df_roadloc_mean = pd.DataFrame(df_['평당가격'].groupby(df_['지역구_도로명']).mean())
    df_roadloc_std = pd.DataFrame(df_['평당가격'].groupby(df_['지역구_도로명']).std())

    df_list = [df_road_mean, df_road_std, df_usageloc_mean, df_usageloc_std,
              df_usagebuilding_mean, df_usagebuilding_std, df_location_mean, df_location_std,df_roadloc_mean,df_roadloc_std]
    col_list = ['road_mean','road_std','usageloc_mean','usageloc_std',
                   'usagebuilding_mean','usagebuilding_std','location_mean','location_std','df_roadloc_mean','df_roadloc_std']
    left_list = ['도로조건','도로조건','용도지역','용도지역',
                    '건축물주용도','건축물주용도','지역구','지역구','지역구_도로명','지역구_도로명']

    for dfl,left,col in zip(df_list,left_list,col_list):
        df_ = df_.merge(dfl, how='left', left_on='{}'.format(left), right_on=dfl.index,
                          suffixes=('','_by_{}'.format(col)))
    
    return df_
df_4 = df_4[df_4.notnull().any(axis=1)]
df_4 = set_cate(df_4)


df=df_4.copy()

NameError: name 'df_test' is not defined

# normalze

In [ ]:
# 연속형 변수 컬럼

cont_col_includeY = ['전용/연면적(㎡)', '층', '건물년식',  '평당가격_by_road_mean',
       '평당가격_by_usageloc_mean', 
       '평당가격_by_usagebuilding_mean', 
       '평당가격_by_location_mean',
       '평당가격_by_df_roadloc_mean', '평당가격']


cont_col = ['전용/연면적(㎡)', '층', '건물년식',  '평당가격_by_road_mean',
       '평당가격_by_usageloc_mean',
       '평당가격_by_usagebuilding_mean',
       '평당가격_by_location_mean',
       '평당가격_by_df_roadloc_mean']


#################################################################################
# TODO: 연속형 변수에 대해서 Z normalize를 설계 및 적용                         #
#################################################################################

X_test, y_test = splitData(df_.loc[:,cont_col_includeY], 5, y_column = '평당가격') 

# 위 block에서 만들어진 z_normalize와 z_normalize_val 적용 (Error 날 경우 다시 고민할 것)
X_test_norm = X_test.copy()
X_test_norm.loc[:, cont_col], cache = z_normalize(X_train, cont_col)
X_val_norm = X_val.copy()
X_val_norm.loc[:, cont_col] = z_normalize_val(X_val, cont_col, cache)



In [ ]:
#################################################################################
# TODO: Normalize : training set을 기준으로 진행                                #
#                                                                               #
# 1. training set의 cache를 활용하여 test set normalize    
# ---- z_normalize_val(____, cache)                           
#################################################################################













In [ ]:
# Print your test 거래금액 Mean Absolute Error: this should be below 11,000

print('Validation MSE(㎡당 가격): ', np.mean(np.square(best_rf.predict(X_test_norm) - y_price)))
print('Validation MAE(㎡당 가격):', np.mean(np.abs(best_rf.predict(X_test_norm) - y_price)))

_price = X_test.loc[:, '전용/연면적(㎡)'] * best_rf.predict(X_test_norm)

print('Validation MSE(거래 금액): ', np.mean(np.square(_price - y_real)))
print('Validation MAE(거래 금액): ', np.mean(np.abs(_price - y_real)))

In [ ]:
def change_type(df_):
def handling_nan(df_):
def remove_outliers(df_):
def cal_days(start, end): #날짜 차이 일(days) 계산용
def parse_road(string):
def new_variable(df_):
